<a href="https://colab.research.google.com/github/gladcolor/Mapillary_image_download/blob/master/mapillary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install packages

In [7]:
! pip install mercantile
! pip install mapbox_vector_tile
! pip install vt2geojson
! pip install haversine


# Input the API key of Mapillary

Create your access token at https://mapillary.com/developer

In [71]:
API_KEY_FILE = r'/content/drive/MyDrive/Research/Mapillary_images/API_KEY.txt'   # put the access_token in the first line.
ACCESS_TOKEN = open(API_KEY_FILE, 'r').readline()
# print("Your access token is:", ACCESS_TOKEN)

# Define functions

In [ ]:
import mercantile, mapbox_vector_tile, requests, json
import pandas as pd
from vt2geojson.tools import vt_bytes_to_geojson
from haversine import haversine
import urllib
import os

def download_img(img_url, basename="downloaded_img.jpg", save_path=os.getcwd()):

    file_full_name = os.path.join(save_path, basename)
    urllib.request.urlretrieve(img_url, file_full_name)
    # return img

def get_image_meta_from_lonlat(ID="", lon=-95.636811, lat=29.587627, zoom=14, filter_radius=30, is_download=False):

    categories_kept = ['object--support--utility-pole','object--street-light'] # keep these categories only

 
    tile = mercantile.tile(lon, lat, zoom)
    output= { "type": "FeatureCollection", "features": [] }
    tile_url = 'https://tiles.mapillary.com/maps/vtp/mly1_public/2/{}/{}/{}?access_token={}'.format(zoom,tile[0],tile[1], ACCESS_TOKEN)
    response = requests.get(tile_url)
                
    data = vt_bytes_to_geojson(response.content, tile.x, tile.y, tile.z)

    # filtered_data = [feature for feature in data['features'] if feature['properties']['value'] in filter_values]

    # print(len(data['features']))
    for feature in data['features']:
        # print(feature['geometry']['type'])
        if feature['geometry']['type'] == 'Point':
            distance = haversine((lon, lat), feature['geometry']['coordinates'], unit="m")
            if distance < filter_radius:
                feature['distance'] = distance
                output['features'].append(feature)
                
    # print(len(output['features']))

    image_list = []

    output_df = pd.json_normalize(output['features'], sep='_')

    output_df['request_lon'] = lon
    output_df['request_lat'] = lat
    output_df['ID'] = str(ID)
    # print()
    output_df['sub_ID'] = output_df['ID'].astype(str) + "_" + [str(i) for i in output_df.index] 

    for idx, row in output_df.iterrows():

        try:
    
            image_id = row['properties_id']
            # print(data['features'][1]['properties'])
        # 
            url = 'https://graph.mapillary.com/{}?fields=id,thumb_2048_url,thumb_1024_url, captured_at,compass_angle,sequence,geometry&access_token={}'.format(image_id, ACCESS_TOKEN)

            response = requests.get(url)
            data1 = response.json()

            output_df.loc[idx, 'thumb_1024_url'] = data1['thumb_1024_url']
            output_df.loc[idx, 'thumb_2048_url'] = data1['thumb_2048_url']

            if is_download:
                img_url = data1['thumb_1024_url']
                download_img(img_url=img_url)
        except Exception as e:
            print("Error in get_image_meta_from_lonlat():", e, ID, lon, lat, feature)
            continue            
    
    return output_df

# Test    -76.97215893969998 38.9310086164
image_meta_df = get_image_meta_from_lonlat(ID=0, lon=-76.9721589396999, lat=38.9310086164, zoom=14, filter_radius=5, is_download=False)
image_meta_df.to_csv('image_meta.csv')
image_meta_df

,type,distance,geometry_type,geometry_coordinates,properties_captured_at,properties_compass_angle,properties_id,properties_is_pano,properties_organization_id,properties_sequence_id,request_lon,request_lat,ID,sub_ID,thumb_1024_url,thumb_2048_url
0,Feature,4.146921,Point,"[-76.9721782207489, 38.930867002531215]",1560886896000,178.000000,174009211315745,False,8.189747e+14,mrqza3zyf8ez4nh4pyvxta,-76.972159,38.931009,0,0_0,https://scontent-ams4-1.xx.fbcdn.net/m1/v/t6/A...,https://scontent-ams4-1.xx.fbcdn.net/m1/v/t6/A...
1,Feature,3.642217,Point,"[-76.97218358516693, 38.93091290558428]",1560886895000,177.700000,318853026469896,True,8.189747e+14,3pqhpbkcn0rscbp5jcuj8s,-76.972159,38.931009,0,0_1,https://scontent-ams4-1.xx.fbcdn.net/m1/v/t6/A...,https://scontent-ams4-1.xx.fbcdn.net/m1/v/t6/A...
2,Feature,2.652791,Point,"[-76.9721782207489, 38.93094628960421]",1565013256500,58.000000,1031687840694946,True,8.189747e+14,f7w99m4m2x5gi6l5iaumls,-76.972159,38.931009,0,0_2,https://scontent-ams4-1.xx.fbcdn.net/m1/v/t6/A...,https://scontent-ams4-1.xx.fbcdn.net/m1/v/t6/A...
3,Feature,2.012161,Point,"[-76.97216212749481, 38.93092959759622]",1578586409184,57.569519,3881187465268412,False,7.914254e+14,yxbf06yx6sdai2eymb5o89,-76.972159,38.931009,0,0_3,https://scontent-ams4-1.xx.fbcdn.net/m1/v/t6/A...,https://scontent-ams4-1.xx.fbcdn.net/m1/v/t6/A...
4,Feature,2.660731,Point,"[-76.9721782207489, 38.93107147953896]",1560801872000,336.900000,680092972778629,False,8.189747e+14,jwxza88nqbxd65d18hg9qv,-76.972159,38.931009,0,0_4,https://scontent-ams4-1.xx.fbcdn.net/m1/v/t6/A...,https://scontent-ams4-1.xx.fbcdn.net/m1/v/t6/A...
5,Feature,3.363923,Point,"[-76.97218894958496, 38.931025576588524]",1560886894000,179.000000,141014774750144,True,8.189747e+14,3pqhpbkcn0rscbp5jcuj8s,-76.972159,38.931009,0,0_5,https://scontent-ams4-1.xx.fbcdn.net/m1/v/t6/A...,https://scontent-ams4-1.xx.fbcdn.net/m1/v/t6/A...
6,Feature,2.930905,Point,"[-76.97218358516693, 38.93096715460868]",1560886894500,179.200000,925164624991907,True,8.189747e+14,3pqhpbkcn0rscbp5jcuj8s,-76.972159,38.931009,0,0_6,https://scontent-ams4-1.xx.fbcdn.net/m1/v/t6/A...,https://scontent-ams4-1.xx.fbcdn.net/m1/v/t6/A...
7,Feature,3.392010,Point,"[-76.97212994098663, 38.93105061456518]",1569053520000,239.140000,491249225448985,False,NaN,mnvGCfKi56IIZ0Do-uNo9w,-76.972159,38.931009,0,0_7,https://scontent-ams4-1.xx.fbcdn.net/m1/v/t6/A...,https://scontent-ams4-1.xx.fbcdn.net/m1/v/t6/A...
8,Feature,1.784955,Point,"[-76.97215139865875, 38.93107147953896]",1565015434500,239.300000,523091835364670,True,8.189747e+14,5xm1sqbjsvahgstfcxg13c,-76.972159,38.931009,0,0_8,https://scontent-ams4-1.xx.fbcdn.net/m1/v/t6/A...,https://scontent-ams4-1.xx.fbcdn.net/m1/v/t6/A...
9,Feature,1.551460,Point,"[-76.97217285633087, 38.931013057596886]",1560801870000,27.600000,993424551196402,False,8.189747e+14,jwxza88nqbxd65d18hg9qv,-76.972159,38.931009,0,0_9,https://scontent-ams4-1.xx.fbcdn.net/m1/v/t6/A...,https://scontent-ams4-1.xx.fbcdn.net/m1/v/t6/A...


# Download images

## Load lon/lat file

In [ ]:
lonlat_file = r'https://github.com/gladcolor/Mapillary_image_download/raw/master/intersection_lonlat.zip'
lonlat_df = pd.read_csv(lonlat_file) 

# clean 
lonlat_df = lonlat_df[['lon', 'lat']].drop_duplicates()
lonlat_df['id'] = lonlat_df.index
lonlat_df

,lon,lat,id
0,-76.969003,38.927727,0
2,-76.961398,38.933604,2
4,-76.970146,38.927727,4
6,-76.968015,38.933001,6
8,-76.965164,38.925176,8
...,...,...,...
67365,-77.014025,38.882372,67365
67384,-77.053307,38.893106,67384
67412,-77.016623,38.822496,67412
67413,-77.016623,38.822496,67413


In [ ]:
def download_image_from_lonlat_df(lonlat_df, save_path=None, file_name_column=None, max_distance=10):
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    meta_df_list = []
    for idx, row in lonlat_df.iterrows():
        lon = row['lon']
        lat = row['lat']

        print("Processing:", idx, lon, lat)

        image_meta_df = get_image_meta_from_lonlat(ID=idx, lon=lon, lat=lat, zoom=14, filter_radius=max_distance, is_download=False)
        # print(lon, lat, len(image_meta_df))
        # print(image_meta_df)

        kept_indx = image_meta_df[['thumb_1024_url']].drop_duplicates().index
        image_meta_df = image_meta_df.loc[kept_indx]

        # print(image_meta_df)

        for idx2, row2 in  image_meta_df.iterrows():
            img_url = row2['thumb_1024_url']
            # print(row2)
            # print(img_url)
            basename = str(row[file_name_column]) + f'_{idx2}.jpg'            
            download_img(img_url, basename=basename, save_path=save_path)

        meta_df_list.append(image_meta_df)
        
        # print(lon, lat)
    all_df = pd.concat(meta_df_list)
    all_df.to_csv('image_meta.csv')

download_image_from_lonlat_df(lonlat_df[100:103], save_path=os.getcwd(), file_name_column='id', max_distance=5)

Processing: 378 -76.97215893969998 38.9310086164
Processing: 398 -76.9648853171 38.9128975915
Processing: 400 -76.9725482038 38.926437358899996


# Download traffic lights

In [62]:
import mercantile, mapbox_vector_tile, requests, json
import pandas as pd
from vt2geojson.tools import vt_bytes_to_geojson
from haversine import haversine
import urllib
import os

def download_img(img_url, basename="downloaded_img.jpg", save_path=os.getcwd()):

    file_full_name = os.path.join(save_path, basename)
    urllib.request.urlretrieve(img_url, file_full_name)
    # return img

def get_features_from_lonlat(ID=0, lon=-95.636811, lat=29.587627, zoom=14, filter_radius=30, target_categories=['object--traffic-light']):

    categories_kept = ['object--support--utility-pole','object--street-light'] # keep these categories only

 
    tile = mercantile.tile(lon, lat, zoom)
    output= { "type": "FeatureCollection", "features": [] }
    # tile_url = 'https://tiles.mapillary.com/maps/vtp/mly1_public/2/{}/{}/{}?access_token={}'.format(zoom,tile[0],tile[1], ACCESS_TOKEN)
    tile_url = 'https://tiles.mapillary.com/maps/vtp/mly_map_feature_point/2/{}/{}/{}?access_token={}'.format(zoom,tile[0],tile[1], ACCESS_TOKEN)
    response = requests.get(tile_url)
                
    data = vt_bytes_to_geojson(response.content, tile.x, tile.y, tile.z)

    # print(data)

    filtered_objects = [feature for feature in data['features'] if feature['properties']['value'] in target_categories]
    print(type(filtered_objects))

    print("objects before filtering:", len(data['features']))
    for feature in filtered_objects:
        # print(feature['geometry']['type'])
        if feature['geometry']['type'] == 'Point':
            distance = haversine((lon, lat), feature['geometry']['coordinates'], unit="m")
            if distance < filter_radius:
                feature['distance'] = distance
                feature['lon'], feature['lat'] = feature['geometry']['coordinates']                
                output['features'].append(feature)
                
    print("objects after filtering:", len(output['features']))

    image_list = []

    output_df = pd.json_normalize(output['features'], sep='_')

    output_df['request_lon'] = lon
    output_df['request_lat'] = lat
    output_df['ID'] = str(ID)
    # # print()
    # output_df['sub_ID'] = output_df['ID'].astype(str) + "_" + [str(i) for i in output_df.index] 

    # for idx, row in output_df.iterrows():

    #     try:
    
    #         image_id = row['properties_id']
    #         # print(data['features'][1]['properties'])
    #     # 
    #         # url = 'https://graph.mapillary.com/{}?fields=id,thumb_2048_url,thumb_1024_url, captured_at,compass_angle,sequence,geometry&access_token={}'.format(image_id, ACCESS_TOKEN)
    #         url = 'https://graph.mapillary.com/{}?fields=id,thumb_2048_url,thumb_1024_url, captured_at,compass_angle,sequence,geometry&access_token={}'.format(image_id, ACCESS_TOKEN)

    #         response = requests.get(url)
    #         data1 = response.json()

    #         output_df.loc[idx, 'thumb_1024_url'] = data1['thumb_1024_url']
    #         output_df.loc[idx, 'thumb_2048_url'] = data1['thumb_2048_url']


    #     except Exception as e:
    #         print("Error in get_image_meta_from_lonlat():", e, ID, lon, lat, feature)
    #         continue            
    
    return output_df

# Test    -76.97215893969998 38.9310086164
# target_categories = ['object--support--utility-pole','object--street-light', 'object--traffic-light']
target_categories = ['object--traffic-light', 
                     'object--traffic-light--pedestrians',
                     'object--traffic-light--general-upright', 
                     'object--traffic-light--general-horizontal', 
                     'object--traffic-light--other', 
                     'object--traffic-light--general-single', 
                     'object--traffic-light--cyclists', 
                     ]

image_meta_df = get_features_from_lonlat(ID=0, lon=-76.9720589396999, lat=38.9200086164, zoom=14, filter_radius=200, target_categories=target_categories)
# image_meta_df.to_csv('image_meta.csv')
image_meta_df

<class 'list'>
objects before filtering: 5035
objects after filtering: 23


,type,distance,lon,lat,geometry_type,geometry_coordinates,properties_first_seen_at,properties_id,properties_last_seen_at,properties_value,request_lon,request_lat,ID
0,Feature,84.361122,-76.972570,38.917520,Point,"[-76.97256982326508, 38.91752047338156]",1544690243147,2944955515774805,1544690243147,object--traffic-light--pedestrians,-76.972059,38.920009,0
1,Feature,60.943946,-76.972108,38.917587,Point,"[-76.97210848331451, 38.91758725401061]",1533931416976,481004343170117,1533931416976,object--traffic-light--pedestrians,-76.972059,38.920009,0
2,Feature,181.231828,-76.970429,38.919862,Point,"[-76.97042942047119, 38.919861931659966]",1544690385343,1132431367241237,1544690385343,object--traffic-light--general-upright,-76.972059,38.920009,0
3,Feature,98.578154,-76.972715,38.917362,Point,"[-76.97271466255188, 38.91736186913562]",1545049699587,2944869822450041,1545049699587,object--traffic-light--general-upright,-76.972059,38.920009,0
4,Feature,97.069777,-76.972677,38.917274,Point,"[-76.97267711162567, 38.917274219268705]",1544690233547,2944817915788565,1544690233547,object--traffic-light--general-upright,-76.972059,38.920009,0
5,Feature,100.323628,-76.972741,38.917391,Point,"[-76.97274148464203, 38.91739108573387]",1533931423579,481076583162893,1533931423579,object--traffic-light--pedestrians,-76.972059,38.920009,0
6,Feature,63.989874,-76.972227,38.917566,Point,"[-76.97222650051117, 38.917566385070785]",1533931417644,480930003177551,1533931417644,object--traffic-light--general-upright,-76.972059,38.920009,0
7,Feature,174.636949,-76.970488,38.919962,Point,"[-76.97048842906952, 38.919962099274755]",1544690386543,1191162224645559,1544690386543,object--traffic-light--general-upright,-76.972059,38.920009,0
8,Feature,197.399294,-76.970285,38.920258,Point,"[-76.97028458118439, 38.920258427640505]",1544775307206,1190570498038065,1544775307206,object--traffic-light--general-upright,-76.972059,38.920009,0
9,Feature,72.105621,-76.972302,38.917341,Point,"[-76.97230160236359, 38.91734100012951]",1419068368000,770360790331508,1489240587000,object--traffic-light--general-upright,-76.972059,38.920009,0


In [73]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import plotly
import plotly.figure_factory as ff
from matplotlib import pyplot as plt

MAPBOX_API_KEY_FILE = r'/content/drive/MyDrive/Research/Mapillary_images/MAPBOX_API_KEY.txt'   # put the access_token in the first line.
mapbox_token = open(MAPBOX_API_KEY_FILE, 'r').readline()

fig = px.scatter_mapbox(image_meta_df, lat="lat", lon="lon",  hover_data=["properties_value"],
                        # center=dict(lat=-26, lon=135), 
                        zoom=13,
                        color_discrete_sequence=["fuchsia"])

fig.update_layout(mapbox_style="satellite-streets",                  
                  mapbox_accesstoken=mapbox_token,        
                  )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, \
                  height=1000)
fig.show()

In [54]:
image_meta_df['properties_value'].unique()

array(['object--traffic-light--general-upright',
       'object--traffic-light--pedestrians',
       'object--traffic-light--general-horizontal',
       'object--traffic-light--general-single'], dtype=object)